# 3. Hafta Lab-2: $L1$ ve $L2$ Regularization Sonuçlarının İncelenmesi
## BGM 565: Siber Güvenlik için Makine Öğrenme Yöntemleri
## İstanbul Şehir Üni. - Bilgi Güvenliği Müh.
### Dr. Ferhat Özgür Çatak
Bu lab çalışmasında lojistik regresyon algoritmasında $L2$ regularization yönteminde $\lambda$ değeriyle ağırlık vektörünün $\mathbf{w}$ değişimi KDDCUP'99 veri kümesinde uygulayacağız.

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
# veri kumesini oku
kolon_adlari = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations',
'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count',
'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
'dst_host_srv_rerror_rate','label']

verikumesi = pd.read_csv("kddcup99.tar.gz",compression="gzip", names=kolon_adlari, 
low_memory=False, skiprows=1)

# ilgili kolonlari sec
secilecek_kolonlar = ['serror_rate','srv_serror_rate',
'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count',
'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
'dst_host_srv_rerror_rate']


X = verikumesi[secilecek_kolonlar].as_matrix()
y = verikumesi['label'].apply(lambda d:0 if d == 'normal.' else 1).as_matrix()

Veri kumesini egitim ve test veri kumesi olarak ayır

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

$\lambda$ değerlerini tanımla

In [4]:
lambdas = [10**-15, 10**-10, 10**-8, 0.0001, 0.001, 0.01, 1, 5, 10, 20, 50, 100, 1000]

Sonuçları kaydetmek için pandas dataframe tanımla.

In [5]:
col = ['lambda','MSE_Train','MSE_Test','w_0'] + ['w_%d'%i for i in range(1,X.shape[1]+1)] 
ind = [i for i in range(1,len(lambdas)+1)]
coef_matrix_simple = pd.DataFrame(index=ind, columns=col)
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Her bir $\lambda$ değeri için $L2$ regularization kullanarak *LogisticRegression* algoritması kullanılarak sınıflandırma modelini olustur.

In [6]:
i = 0
for l in lambdas:
    clf = LogisticRegression(penalty='l2', C=1/l)
    clf.fit(X_train,y_train)
    
    y_hat_train = clf.predict(X_train)
    mse_train = mean_squared_error(y_train, y_hat_train)
    
    y_hat_test = clf.predict(X_test)
    mse_test = mean_squared_error(y_test, y_hat_test)
    
    coef_matrix_simple.iloc[i,0] = l
    coef_matrix_simple.iloc[i,1] = mse_train
    coef_matrix_simple.iloc[i,2] = mse_test
    coef_matrix_simple.iloc[i,3] = clf.intercept_[0]
    coef_matrix_simple.iloc[i,4:] = clf.coef_
    i += 1
    print('Tamamlanan : % {0}'.format(i/len(lambdas)*100), end = '\r',flush=False)
    #os.system('cls' if os.name=='nt' else 'clear')
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,1e-15,0.0180668,0.0178559,-5.97812,4.3598,2.38185,-2.48476,2.56381,-7.36867,-2.57685,...,0.0390847,-0.0168822,5.26454,-7.57855,7.54041,7.22619,5.04505,2.94285,3.09141,1.79822
2,1e-10,0.0180004,0.0177455,-4.5669,3.17107,2.08508,-0.700424,1.55699,-7.12767,-1.19907,...,0.031908,-0.0124549,4.2927,-6.99137,7.39696,3.87942,3.38101,2.28354,1.76188,1.36775
3,1e-08,0.0179974,0.0177455,-4.56684,3.1712,2.08501,-0.700661,1.55697,-7.12761,-1.19828,...,0.0319053,-0.0124529,4.29282,-6.99077,7.39682,3.87939,3.38115,2.28348,1.76173,1.36773
4,0.0001,0.0179974,0.0177394,-4.56691,3.17107,2.08507,-0.700434,1.55699,-7.12766,-1.199,...,0.0319108,-0.0124581,4.29273,-6.99132,7.39694,3.87942,3.38101,2.28353,1.76186,1.36774
5,0.001,0.0180004,0.0177455,-4.56688,3.17106,2.08506,-0.70045,1.55698,-7.12764,-1.19905,...,0.0319023,-0.0124486,4.29272,-6.99134,7.39692,3.87945,3.38101,2.28352,1.76188,1.36774
6,0.01,0.0180004,0.0177455,-4.5668,3.17093,2.0849,-0.70057,1.55698,-7.12738,-1.19923,...,0.0319067,-0.0124549,4.29282,-6.9913,7.39664,3.8797,3.38089,2.28337,1.76202,1.36773
7,1,0.0180276,0.0177885,-4.55698,3.15686,2.06804,-0.713637,1.55673,-7.10001,-1.22197,...,0.031882,-0.0125644,4.30118,-6.98935,7.36731,3.90754,3.36807,2.26775,1.77799,1.36663
8,5,0.0184445,0.0182669,-4.93311,3.25774,2.00507,-1.03988,1.79012,-6.84365,-1.85726,...,0.0332541,-0.0135297,4.43045,-7.10116,7.26832,4.51323,3.57106,2.27971,2.15748,1.48457
9,10,0.0181575,0.0179479,-4.60083,3.03369,1.8688,-0.925444,1.60439,-6.73428,-1.42906,...,0.0311687,-0.0122814,4.21167,-6.67347,7.11846,4.19282,3.28078,2.09721,1.97587,1.37038
10,20,0.0184808,0.0182056,-4.35214,2.60879,1.69069,-0.60355,1.42059,-6.42725,-1.31223,...,0.0292089,-0.0102481,3.69843,-6.32535,6.94735,3.55272,2.7968,1.87057,1.82452,1.24571


Her bir $\lambda$ değeri için $L1$ regularization kullanarak *LogisticRegression* algoritması kullanılarak sınıflandırma modelini olustur.

In [7]:
i = 0
for l in lambdas:
    clf = LogisticRegression(penalty='l1', C=1/l)
    clf.fit(X_train,y_train)
    
    y_hat_train = clf.predict(X_train)
    mse_train = mean_squared_error(y_train, y_hat_train)
    
    y_hat_test = clf.predict(X_test)
    mse_test = mean_squared_error(y_test, y_hat_test)
    
    coef_matrix_simple.iloc[i,0] = l
    coef_matrix_simple.iloc[i,1] = mse_train
    coef_matrix_simple.iloc[i,2] = mse_test
    coef_matrix_simple.iloc[i,3] = clf.intercept_[0]
    coef_matrix_simple.iloc[i,4:] = clf.coef_
    i += 1
    print('Tamamlanan : % {0}'.format(i/len(lambdas)*100), end = '\r',flush=False)
    #os.system('cls' if os.name=='nt' else 'clear')
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,1e-15,0.0150426,0.0152613,-8.97598,9.20642,-5.6218,-16.8529,13.2003,-5.82757,-1.77567,...,0.0452344,-0.0217352,6.35028,-8.06092,7.62742,14.455,6.8078,22.8707,5.60005,2.92346
2,1e-10,0.0150668,0.0152797,-8.68449,9.15552,-5.60667,-16.7955,13.1544,-5.99088,-1.9091,...,0.0447633,-0.0213591,6.24916,-8.04836,7.61655,14.2837,6.79001,22.8109,5.54078,2.92467
3,1e-08,0.0150456,0.0152735,-8.92095,9.18061,-5.58775,-16.85,13.196,-5.87158,-1.8093,...,0.0451844,-0.0217267,6.34984,-8.05973,7.62774,14.4367,6.80495,22.8371,5.5897,2.92704
4,0.0001,0.0150607,0.0152613,-8.76659,9.14392,-5.57203,-16.8304,13.1787,-5.95548,-1.8787,...,0.044931,-0.0215081,6.28963,-8.05396,7.62157,14.3458,6.80222,22.8015,5.56596,2.92505
5,0.001,0.0150426,0.0152551,-9.02606,9.23142,-5.62829,-16.8166,13.1685,-5.80505,-1.75548,...,0.045325,-0.0218335,6.37984,-8.05776,7.6288,14.4879,6.81031,22.842,5.60372,2.92181
6,0.01,0.0150366,0.0152306,-9.15318,9.20381,-5.59173,-16.8482,13.186,-5.73568,-1.6958,...,0.0455193,-0.0219809,6.42471,-8.0563,7.63285,14.56,6.82102,22.8286,5.62945,2.929
7,1,0.0160547,0.0161998,-8.85637,6.71123,-2.49121,-16.4568,12.9237,-5.91612,-1.74465,...,0.0449847,-0.0216106,6.36001,-7.97975,7.60205,14.2867,6.96911,19.4306,5.51574,2.89347
8,5,0.0162511,0.0164083,-8.98648,4.6927,0,-14.1367,11.2397,-5.75177,-1.43032,...,0.0446635,-0.0214574,6.39427,-7.81531,7.54638,13.786,6.90595,14.6371,5.36356,2.44812
9,10,0.01652,0.0165433,-8.28848,4.26542,0,-11.2858,9.30096,-6.00888,-1.48448,...,0.0429293,-0.0201262,6.08895,-7.65208,7.46356,12.6694,6.57765,11.8259,5.00415,1.77975
10,20,0.0174415,0.0172732,-7.63079,3.95981,0,-6.41251,6.22692,-6.13221,-1.31328,...,0.0407619,-0.0185604,5.75476,-7.37822,7.34276,10.9223,6.05455,7.18622,4.4035,0.44197


Pandas dataframe ile **SQL** benzeri sorgular yapabilirsiniz.

In [8]:
coef_matrix_simple.query("w_2 == 0 and w_3 == 0")

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
11,50,0.0182179,0.0179602,-6.42318,3.5069,0,0,1.22533,-5.86126,-0.650602,...,0.035012,-0.014206,4.71889,-6.86839,7.12609,7.24679,5.03008,1.45258,3.08247,0
12,100,0.018943,0.0187822,-4.31372,2.07044,0,0,0.670075,-5.98698,-0.332615,...,0.027905,-0.00886217,3.23177,-6.5648,6.93103,3.62298,3.9164,0.496872,2.97818,0
13,1000,0.0292937,0.0290811,0,0.272684,0,0,0,-6.0496,0,...,0.0154434,0.0018056,0,-3.7373,5.59601,0,0.804203,0,0,0
